In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.cluster import KMeans

In [ ]:
!pip install pythainlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd

In [ ]:
from pythainlp import sent_tokenize, word_tokenize

In [ ]:
docs = pd.read_csv('/content/Open source [WeVis-Promise Tracker] Data - promise.csv')

In [ ]:
docs['promiseTitle'].values

array(['1 จังหวัด 1 แหล่งท่องเที่ยว',
       '1 ตำบล 1 ทุน ทุนการศึกษาจบปริญญาตรีฟรี มีงานทำ',
       '1 ตำบล 1 ธนาคาร', '30 บาทรักษาทุกโรคได้จริง', '30 รักษาทั่วไทย',
       'LGBT สิทธิต้องเท่าเทียม\n\n', 'Single mom 10,000 บาทต่อเดือน',
       'Street food จัดระเบียบและหาสถานที่ทดแทนให้ร้านค้า',
       'Work@home ทำงานที่ไหนก็ได้ เงินเดือนเท่าเดิม',
       'กระจายรายได้ส่วนกลางต่อท้องถิ่น',
       'กระจายอำนาจอธิปไตยให้ชุมชนท้องถิ่น',
       'กองทุนกู้ยืมเพื่อการศึกษาระดับมหาวิทยาลัยโดยผู้กู้ยืมค้ำประกันตัวเองได้',
       'กัญชา 1,000,000 ไร่',
       'กัญชาไทย ปลูกได้เสรี และนำไปใช้ทางการแพทย์',
       'การเข้าถึงสินเชื่อดอกเบี้ยต่ำเพื่อที่อยู่อาศัยผู้มีรายได้น้อย',
       'การค้าออนไลน์อาชีพสงวนคนไทย',
       'การจัดตั้งกองทุนสวัสดิการเพื่อเกษตรกร เริ่มต้นที่วงเงิน 20,000 ล้านบาท',
       'การปฏิรูปพลังงาน',
       'การวางผังเมืองใหม่ให้กลายเป็น Green and Clean City',
       'ก้าวข้ามความขัดแย้ง เปลี่ยนแปลงปฏิรูปแผ่นดิน',
       'กำจัดอุปสรรคในการทำธุรกิจ',
       'เกษตรกรปลอดหนี้ใ

In [ ]:
vectorizer = TfidfVectorizer(tokenizer=word_tokenize)
doc_matrix = vectorizer.fit_transform(docs['promiseTitle'].values)

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
doc_matrix

<329x1062 sparse matrix of type '<class 'numpy.float64'>'
	with 3014 stored elements in Compressed Sparse Row format>

In [ ]:
clusterer = KMeans(n_clusters=12, verbose=True)

In [ ]:
clustered_docs = clusterer.fit_predict(doc_matrix)

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Initialization complete
Iteration 0, inertia 528.3001996039711.
Iteration 1, inertia 292.9696443100953.
Iteration 2, inertia 292.45709585430734.
Iteration 3, inertia 292.2221673232526.
Converged at iteration 3: strict convergence.
Initialization complete
Iteration 0, inertia 532.3108797666118.
Iteration 1, inertia 293.33166686755743.
Iteration 2, inertia 292.6665008162469.
Iteration 3, inertia 292.3694707623981.
Iteration 4, inertia 292.21608050026424.
Iteration 5, inertia 292.15859795829556.
Converged at iteration 5: strict convergence.
Initialization complete
Iteration 0, inertia 534.9226723144849.
Iteration 1, inertia 293.7844267321627.
Iteration 2, inertia 292.6118797854052.
Iteration 3, inertia 292.2498268891301.
Iteration 4, inertia 292.17815541104994.
Converged at iteration 4: strict convergence.
Initialization complete
Iteration 0, inertia 531.7163891095777.
Iteration 1, inertia 294.4264566612056.
Iteration 2, inertia 293.60710217032556.
Iteration 3, inertia 292.68822110071324.

In [ ]:
clustered_docs

array([ 8,  8,  8,  8,  4,  6,  0,  6,  5,  1,  1,  2,  2,  4, 11, 11,  2,
       11,  6, 11, 11,  0,  2, 11,  6,  6, 11,  6,  1,  6,  6,  6,  7,  7,
        0,  7,  7,  7, 11,  0,  0,  0,  0,  6,  0,  4,  6,  0,  5,  4,  8,
        3,  2,  2,  0,  0,  8,  2,  0,  0, 11,  0, 11,  0,  0,  7,  8,  2,
        2,  8,  1,  9,  6, 10, 11, 11,  4, 11,  5,  2, 11,  1,  0,  6,  0,
        1,  6,  6,  4,  2,  9,  9,  2,  8,  2, 10, 11,  4, 11,  4, 11,  5,
        7,  0,  8,  0,  2,  0,  4,  4,  4,  8, 11,  3,  0,  2,  2,  2,  2,
        6,  1,  0,  0, 11, 11,  6, 11, 10, 10, 10,  6,  6, 11,  6,  4,  6,
        5,  0,  0,  0,  2, 11,  6,  2, 11,  2,  4,  0,  8,  0,  6,  0,  2,
        2,  3, 11,  3,  1,  2,  0, 11,  6,  0,  5,  0,  0,  0,  6, 11,  6,
        0,  0, 11, 10,  6, 11,  4,  2,  2,  0, 11,  2,  6,  6,  6,  0, 10,
       11,  3,  0,  8, 10, 10, 10, 10, 10,  1,  1,  1,  1,  2,  5, 10,  0,
        8,  6,  1,  9,  2,  0,  6, 11,  1,  0,  8,  3,  6,  0, 11,  8,  1,
        6,  6,  3,  6,  6

In [ ]:
import numpy as np

In [ ]:
newdf = pd.DataFrame(columns=['data','label_clusters'],data=np.array([docs['promiseTitle'].values,clustered_docs]).T)

In [ ]:
all_df = newdf[newdf['label_clusters'] == 0]
for i in range(1,13):
  all_df = all_df.append(newdf[newdf['label_clusters'] == i])

In [ ]:
all_df.to_csv('mk.csv',index=False)